<a href="https://colab.research.google.com/github/SiwawawaXD/LLMclassification/blob/main/pythonCallChatgpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install -q openai

In [2]:
import openai
from google.colab import userdata

openai.my_api_key = userdata.get('API_KEY')
messages = [ {"role": "system", "content": "You are a intelligent assistant."} ]

In [ ]:
while True:
    message = input("User : ")
    if message:
        messages.append(
            {"role": "user", "content": message},
        )
        chat = openai.ChatCompletion.create(
            model="gpt-3.5-turbo", messages=messages
        )

    reply = chat.choices[0].message.content
    print(f"ChatGPT: {reply}")
    messages.append({"role": "assistant", "content": reply})